<a href="https://colab.research.google.com/github/cerezamo/NLP_brouillon/blob/master/Camembert_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CamemBERT classification model 


In [53]:
import spacy 
import numpy as np 
import pandas as pd 
import os 
os.getcwd()
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Set up Colab GPU 

In [54]:
#%tensorflow_version 1.x
# First you should go in 'Edit' -> 'Notebook settings' -> Add device GPU
import tensorflow as tf


# GPU device name.
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

Let's now tell torch that one GPU is available 

In [55]:
import torch

if torch.cuda.is_available():  
        
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P4


Please check GPU capacity that you were given. You might want to reduce the batch size further in the code. 

Let's install the Hugging Face Library transformer package 

In [56]:
! pip install transformers 

### Loading our corpus and preprocessing 

In [0]:
import pandas as pd
# Import medium_df_desq in "files"
# Load the dataset into a pandas dataframe.
#df=pd.read_csv('medium_df_deseq.csv',encoding='utf-8')
df=pd.read_csv('medium_df_deseq.csv',encoding='utf-8')

# We replace the labels in a more normalized way : 0=men, 1=women 
df.sexe=df.sexe.replace(1,0)
df.sexe=df.sexe.replace(2,1)


In [0]:
# Make results reproducible 
seed_val = 2020

In [0]:
def unbalanced_preprocess(df,seed_val,frac_val):

  from sklearn.model_selection import train_test_split

  #Shuffle the data 
  df_unbalanced=df.sample(frac=frac_val).reset_index()

  # Reduce to the variables we are interested in 
  df_unbalanced=df_unbalanced[['Texte','sexe']]

  # Report the number of speeches in the corpus.
  print('Number of text in the unbalanced corpus : {:,}\n'.format(df_unbalanced.shape[0]))
  prop = (len(df_unbalanced[df_unbalanced.sexe==1])/len(df_unbalanced))*100
  print('Proportions of women in the unbalanced corpus : {}\n'.format(prop))

  # We keep one little sample for evaluation 
  model_unbalanced, dev_unbalanced = train_test_split(df_unbalanced, test_size=0.02,random_state=seed_val)

  return model_unbalanced, dev_unbalanced

In [60]:
df_unbalanced = unbalanced_preprocess(df,seed_val,frac_val=0.5)[0]

Number of text in the unbalanced corpus : 2,500

Proportions of women in the unbalanced corpus : 25.28



In [0]:
def balanced_preprocess(df,seed_val,frac_val):
  from sklearn.model_selection import train_test_split

  # Let's take a balanced sample 
  df_m = df.loc[df['sexe'] == 0]
  df_f = df.loc[df['sexe'] == 1] 
  df_m = df_m[0:len(df_f)]
  df = df_f.append(df_m)

  #Shuffle the data and taking half of the sample in order not to have to many data compared to the other samples 
  df_balanced=df.sample(frac=1).reset_index()

  # Reduce to the variables we are interested in 
  df_balanced=df_balanced[['Texte','sexe']]

  # Report the number of speeches in the corpus.
  print('Number of text in this corpus : {:,}\n'.format(df_balanced.shape[0]))
  prop = (len(df_balanced[df_balanced.sexe==1])/len(df_balanced))*100
  print('Proportions of women in the balanced corpus : {}\n'.format(prop))

  # Keep one little sample for evaluation 
  model_balanced, dev_balanced = train_test_split(df_balanced, test_size=0.02,random_state=seed_val)

  return model_balanced, dev_balanced



In [62]:
df_balanced = balanced_preprocess(df,seed_val,frac_val=1)[0]

Number of text in this corpus : 2,500

Proportions of women in the balanced corpus : 50.0



In [0]:
def balanced_splitted(df,seed_val,frac_val):
  from sklearn.model_selection import train_test_split
    # Reduce to the variables we are interested in 
  df=df[['Texte','sexe']]

    # Let's take a balanced sample 
  df_m = df.loc[df['sexe'] == 0]
  df_f = df.loc[df['sexe'] == 1] 
  df_m = df_m[0:len(df_f)]
  df = df_f.append(df_m)

   #Shuffle the data
  df=df.sample(frac=frac_val).reset_index()

  from itertools import repeat

  n=2500
  chunks, label_split=[],[]
  j=0
  for text in df.Texte :
      txt=[text[i:i+n] for i in range(0, len(text), n)]
      chunks.append(txt)
      label_split.extend(repeat(df.sexe[j], len(txt)))
      j+=1

  chunks = [item for sublist in chunks for item in sublist]
  df=pd.DataFrame([chunks,label_split]).transpose()
  df.columns=['Texte','sexe']

  # Put as integer 
  df['sexe'] = df['sexe'].astype(int)

  df_balanced_split= df

  # Report the number of speeches in the corpus.
  print('Number of text in this balanced splitted corpus : {:,}\n'.format(df_balanced_split.shape[0]))
  prop = (len(df_balanced_split[df_balanced_split.sexe==1])/len(df_balanced_split))*100
  print('Proportions of women in the balanced splitted corpus : {}\n'.format(prop))

  # Keep one little sample for evaluation 
  model_balanced_split, dev_balance_split = train_test_split(df_balanced_split, test_size=0.02,random_state=seed_val)
  
  return model_balanced_split, dev_balance_split

In [64]:
df_balanced_split = balanced_splitted(df,seed_val,0.2)[0]

Number of text in this balanced splitted corpus : 2,531

Proportions of women in the balanced splitted corpus : 53.299091268273415



**We propose 3 samples to train our model :**


1.   **Unbalanced sample**

We take the raw data without any further treatment.

2.   **Balanced sample**

The second option consist in deleting randomly part of male speeches in order to get a balanced sample. Indeed, in the case of unbalanced sample our model could decide to classify all speakers in the male category which would lead to a 0.75 accuracy in our case study. In order to avoid this we feed the model with the same proportions of male and female speakers. Other kind of treatments exist to deal with unbalanced sample. This one is the simpliest one and we could argue that there is a possibility that the deleted sample contains important information that we therefore miss. However we believe that in our case this is not a big issue. Our unbalanced sample is quite large for both female and male.

3. **Balanced and splitted sample**

The third option is a response to the max length constraint of BERT models. Our text samples are big and contain much more tokens than the 512 limit. In the first two options we decide to just feed the model with the 512 first tokens and thus delete the rest of them. In this third option we cut the text into x parts containing 500 tokens each. All parts of the speech will serve to feed the model. By this technique we do not loose potential important informations at the end of the text. A lot of other techniques have been employed (see ref !!! PUT). We decide to stick to this one in this project. 



#### Tokenization of our text and preparing to feed CamemBERT

#### Loading the Camembert Tokenizer

In [0]:
# Import Camembert tokenizer
from transformers import CamembertTokenizer
# We choose a right padding side for the moment and we will test for a left padding side on a second stage
tokenizer = CamembertTokenizer.from_pretrained('camembert-base', do_lower_case=False,padding_side='right') #left

In [66]:
# Print the original text.
print(' Original: ', df.Texte[0])

# Print the text split into tokens.
print('Tokenized: ', tokenizer.tokenize(df.Texte[0]))

# Print the text mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df.Texte[0])))

 Original:  Messieurs,Je suis heureux de vous saluer. Quand je dis que je suis heureux de vous saluer, ce n'est pas une simple affirmation de politesse. Je le disais à l'instant à Monsieur le Ministre de la Défense, M. Richard, c'est pour moi un instant où il y a un peu d'émotion; je vais vous dire pourquoi.Vous êtes la première classe d'âge qui ne fera pas de service militaire. C'est une décision que j'ai prise, il y a deux ans, après une vraie réflexion et un vrai débat. Après tout, le service militaire c'est une vieille tradition nationale, il était plus que centenaire. Il y avait toutes sortes de raisons à cela, notamment la nécessité d'avoir une armée nombreuse et donc d'avoir des jeunes formés aux combats, à l'utilisation des armes de l'époque.On pouvait s'interroger sur la nécessité de poursuivre dans cette voie. Il y avait naturellement des critiques, il y avait beaucoup de jeunes qui se disaient qu'ils perdaient un peu leur temps, d'autres qui étaient satisfaits. Mais il y ava

#### Adding special tokens to the start and end of the text


Preprocessing steps : 


1.   **Add special tokens [CLS] [SEP]** 

According to the documentation we need to add special tokens to the start and end of the text Moreover, for camembert we should add a space between CLS and the first token (not sure here, we have to ask benjamin). 

2.   **Pad and truncate all texts to a single number**

Pretrained transformes like Camembert only accept input of the same length. Our corpus contains large texts and we have to pad them in order to be able to feed Camembert. We will set the max length to a large number in order to get all information possible in the text. We choose a max length of 500 which is almost the maximum (512) "sentence" length  accepted. We are aware that this choice will impact a lot training speed.

3.   **Construct an attention mask**

Attention masks are just set to 1 when the token have to be analyzed and 0 otherwise (padded tokens). All our attention mask should be 1 with this corpus. 



For sake of simplicity and to avoid errors we will use the function encode_plus of the library which is really convenient. 



#### Length and attention mask 

In [0]:
def prepare_to_feed(df,length,batch_size_value):
  from torch.utils.data import TensorDataset, random_split
  from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

  texts = df.Texte.values
  labels = df.sexe.values

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []
  num_truncated_tokens =[]
  # Apply function to our corpus
  for text in texts:
      encoded_dict = tokenizer.encode_plus(
                          text,                      # text
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = length,           # We choose for now a max length of 500.
                          pad_to_max_length = True,    # Pad text to max (marche pas en pad left ?)
                          return_attention_mask = True,   # Construct attention masks
                          return_tensors = 'pt',     # Return pytorch tensors.
                          return_overflowing_tokens =True, # return overflowing token information
                    )
      
      # Map tokens to their id in the dictionnary 
      # We add this to our list    
      input_ids.append(encoded_dict['input_ids'])
  
      #num_truncated_tokens.append(encoded_dict['num_truncated_tokens'])
      
      # 3. Attention masks
      attention_masks.append(encoded_dict['attention_mask'])

  # We convert all this into tensors in order to be able to make it work on GPU 
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Original text and transformed tensor print 
  print("Let's check for the first text indexes, attention masks and labels")
  print(" ")
  print('Original: ', texts[0][0:100])
  print('IDs:', input_ids[0][0:100])
  print('Attention masks:', attention_masks[0][0:100])
  print('labels',labels[0])


  # Combine all above
  dataset = TensorDataset(input_ids, attention_masks, labels)

  # Let's create a 80-20 train / validation dataset 
  train_size = int(0.8 * len(dataset))
  val_size = len(dataset) - train_size

  train_set, val_set = random_split(dataset, [train_size, val_size])


  print("-------------------------------------------------")
  print(" ")
  print("How many texts do we have in the train and validation sample ? ")
  print(" ")
  print('We have {} training texts'.format(train_size))
  print('We have {} validation texts'.format(val_size))
  print(" ")
  print("-------------------------------------------------")

  # We set the size of the batch ( usually set around 16 or 32), we will take the lower bound because of the large text length
  batch_size = batch_size_value

  # We create data loaders for the train and validation dataset. 
  train_dataloader = DataLoader(
              train_set,  # The training samples.
              sampler = RandomSampler(train_set), # Select batches randomly
              batch_size = batch_size # Trains with this batch size.
          )

  val_dataloader = DataLoader(
              val_set, # The validation samples.
              sampler = SequentialSampler(val_set), # Pull out batches sequentially.
              batch_size = batch_size # Evaluate with this batch size.
          )
  
  print('Data loaders created for train [0] and val [1]')

  return train_dataloader, val_dataloader 

In [17]:
print('############### Unbalanced sample ################')
train_loader_unbalanced, val_loader_unbalanced = prepare_to_feed(df_unbalanced,length=500,batch_size_value=2)

############### Unbalanced sample ################
Let's check for the first text indexes, attention masks and labels
 
Original:  Monsieur le Préfet, Monsieur le Président du Conseil régional, Monsieur le Président de l'Associatio
IDs: tensor([    5,  2445,    16, 25230,     7,  2445,    16,  1850,    25,   960,
         4124,     7,  2445,    16,  1850,     8,    17,    11,  3969,    20,
         4678,    10,     8,   184,     7, 19923,    19,  1850,    10,     8,
         4678,    10,     7,  4348,    13, 28499,    35,     8,    13, 19641,
         1467,     7, 23605,    14, 15867,    19, 12680,     7, 23605,    14,
        15867,    19,  6664,  3980,     7,  6684,    10,  9402,    80,    14,
        15867,    19,  3454,     7, 23605,    14, 19923,     7,   228,    11,
           41,    42,   217,     8,   593,    27,    50,   146,   405,    11,
          265,    42,    39,    15,  5155,     7,    24,    16,  9280,     8,
         3864,     8,    13,  4732,  1230,  8820,  1461,    2

In [18]:
print('############### Balanced sample ################')
train_loader_balanced, val_loader_balanced = prepare_to_feed(df_balanced,length=500,batch_size_value=2)

############### Balanced sample ################
Let's check for the first text indexes, attention masks and labels
 
Original:  Mesdames et Messieurs les Ministres,Monsieur le Chef de l'opposition,Mesdames et Messieurs les Secré
IDs: tensor([    5, 23605,    14, 19923,    19,  6375,    10,     7, 31609,    16,
         6238,     8,    17,    11,  4244,     7,  6684,    10,  9402,    80,
           14, 19923,    19, 11907,    10,  2348,     7, 31609,    16,  4860,
           26,  7383,  4975,    10,  1114,  4681,     7,  3181,  9402,    35,
           13,  6238,     8,  9013,     8,    13,  6854,     8,    13,  2358,
         1467,     7, 31609,    16,  1850,    25,   960,  4124,     8,    13,
         6873,     7,    14,  4348,  3181,  9402,    35,    14,  2445,    19,
        15538,    26, 22778,    10,    25,   960,  4124,     7, 31609,    17,
           11, 16336,   297,     7,  4212,  2242,  2650, 20112,  4613,    10,
            7,  3696,    10, 12784,     7,  1684,    45,    52,

In [19]:
print('############### Balanced sample split ################')
train_loader_balanced_split, val_loader_balanced_split = prepare_to_feed(df_balanced_split,length=500,batch_size_value=2)

############### Balanced sample split ################
Let's check for the first text indexes, attention masks and labels
 
Original:  tions un dispositif permettant l'équilibre entre des exigences de sûreté particulièrement élevées et
IDs: tensor([    5,    21,  6279,    23,  2507,  1177,    17,    11,  4786,   128,
           20,  3768,     8, 15850,    21,   937, 12187,    14,    19,   979,
           20,   699,     9,   228,    11,    41,   590,     7,    50,   441,
           27,    19, 27696,    10,    31,    56,  5989,     8,  1222,    29,
           19,  2001,  6793, 16671,   245,     8,   242, 12117, 15910,    80,
            9,  5076,  2170,    50,   920,    27,    17,    11,    88,   218,
          119,   756,    36,   533,     8,    13, 15850, 14248, 13895,     9,
          100,   555,    27,    50,   672,  1733,    32,    39,    14,    27,
           39,   978,   138,  5205,   131,    15,   129,  7030,    24,    22,
        15881,     7,    29,    19,  6511,   274,     7, 

5 and 6 seem to be the [CLS] and [SEP] special tokens 


### CamemBERT Sequence Classification model tuning


#### Loading the model


We will finally build up our model. We will use the  CamemBERT model for sequence classification which includes a special top layer designed for this task. 

In [0]:
# Importing from transformers
from transformers import CamembertForSequenceClassification, CamembertConfig

In [0]:
# Loading the model
gender_model = CamembertForSequenceClassification.from_pretrained(
    "camembert-base", 
    num_labels = 2, # We have two different labels Women = 1 and Men =0   
    output_attentions = False, 
    output_hidden_states = False, 
)

In [70]:
# We run the model on the colab GPU 
gender_model.cuda()

RuntimeError: ignored

Optimizers and Loss

#### Constructing the training and validation loop 


In [0]:
import numpy as np

from sklearn.metrics import f1_score 

from sklearn.metrics import roc_auc_score 

from sklearn.metrics import confusion_matrix

from sklearn.metrics import classification_report

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def flat_f1_score(labels,preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,pred_flat,zero_division=1)

def flat_roc_auc(labels,preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return roc_auc_score(labels_flat,pred_flat,zero_division=0.5)


In [0]:

gender_model = CamembertForSequenceClassification.from_pretrained(
    "camembert-base", 
    num_labels = 2, # We have two different labels Women = 1 and Men =0   
    output_attentions = False, 
    output_hidden_states = False, 
)

In [0]:
def train_val_gendermodel(train_loader, val_loader, epochs_val,seed_val,device,lr_value):

  ############################  IMPORT MODEL ################################################
  gender_model = CamembertForSequenceClassification.from_pretrained(
    "camembert-base", 
    num_labels = 2, # We have two different labels Women = 1 and Men =0   
    output_attentions = False, 
    output_hidden_states = False, )

  model = gender_model
  model.cuda()
  
  ############################## RANDOM SEED ##################################################

  import random
 # Let's put a seed to make this result reproducible 
  seed=seed_val
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)

  ############################### LEARNING RATE SCHEDULER #######################################

  # https://huggingface.co/transformers/migration.html 
  # https://pytorch.org/docs/stable/optim.html (default values)

  import torch.nn as nn
  import torch.optim as optim
  from transformers import AdamW
  from transformers import get_linear_schedule_with_warmup

  epochs = epochs_val # In order to fine tune our model we will first set the number of epochs to 4.

  # We choose Binary cross enthropy with logits loss for the loss computation. It seems to be the most adapted loss to our problem. 
  criterion = nn.BCEWithLogitsLoss()
  #Implements Adam algorithm with weight decay fix.
  opti = AdamW(model.parameters(),
                    lr =lr_value, # learning rate (default = 1e-3)
                    eps = 1e-8 # prevents division by 0 (default = 1e-8)
                  )

  num_training_steps = len(train_loader) * epochs
  # Create a schedule with a learning rate that decreases linearly after linearly increasing during a warmup period (0 here)
  scheduler = get_linear_schedule_with_warmup(opti, 
                                              num_warmup_steps = 0,
                                              num_training_steps = num_training_steps)
    

  # We want to evaluate the training phase 
  training_stats = []

  for ep in range(0, epochs):
    print('===========Starting Epoch {} / {} =============='.format(ep+1,epochs))
    print('Training starts')

    ################################### TRAINING ################################

    #Put the model in training mode
    model.train()

    # Set the train loss for the epoch to 0 
    total_train_loss = 0

    for step, batch in enumerate(train_loader):
      # Clear gradients 
      model.zero_grad() # (opti.zerograd ? )

      # Cpy the 3 batch to GPU 
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      
      #return loss and logits
      loss, logits = model(b_input_ids, 
                          token_type_ids=None, 
                          attention_mask=b_input_mask, 
                          labels=b_labels) 
      
      # Accumulate training loss for all batches 
      total_train_loss += loss.item()

      #Backpropagating the gradients 
      loss.backward()

      # Prevent exploding gradients problem  (forcing the gradients to be small, the parameter updates will not push the parameters too far from their previous values)
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Update parameters 
      opti.step()

      # Update learning rate schedule
      scheduler.step()

    #Calculate the average training loss over all batches  
    avg_train_loss = total_train_loss / len(train_loader)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print('')
    print('And now, validation STARTS')

    ###################### VALIDATION #############################

    # Put model in evaluation mode 
    model.eval()

    # Set statistics to 0
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    total_eval_f1=0
    total_roc_auc = 0 

    # Confusion matrix ?
    predictions, true_labels = [], []

    for batch in val_loader:
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      
      # We don't care about gradients for eval

      with torch.no_grad(): 
        (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
      total_eval_loss += loss.item()

        # Move logits and labels to CPU 
      logits = logits.detach().cpu().numpy()
      label_ids = b_labels.to('cpu').numpy()

      pred_flat = np.argmax(logits, axis=1).flatten()
      labels_flat = label_ids.flatten()

      # Confusion matrix ?
      #val_batch_preds = np.argmax(logits, axis=1)
      #val_batch_labels = label_ids
      #predictions.extend(val_batch_preds)
      #true_labels.extend(val_batch_labels)

      # Accumulation accuracy for all batch
      total_eval_accuracy += flat_accuracy(logits, label_ids)

      # Accumulation f1 for all batch
      total_eval_f1 += f1_score(labels_flat,pred_flat,zero_division=1)

      # Accumulation roc_auc for all batch
      #total_roc_auc += flat_roc_auc(label_ids,logits)
      
      #Final accuracy on all batch
    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      #Final f1 on all batch
    avg_val_f1 = total_eval_f1 / len(val_loader)
    print("  Accuracy: {0:.2f}".format(avg_val_f1))

     #Final roc_auc over all batch
    #avg_val_roc_auc = total_roc_auc / len(val_loader)
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))

      #Final loss over all batch
    avg_val_loss = total_eval_loss / len(val_loader)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))

    # confusion matrix ? 
    pred_tags = [i for i in predictions]
    valid_tags = [i for i in true_labels]



    training_stats.append(
          {
              'epoch': ep + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Valid F1' : avg_val_f1,
             # 'Valid ROC AUC' : avg_val_roc_auc,
          }
      )

  print("")
  print("Done !")

  return  training_stats

In [26]:
results_unbalanced = train_val_gendermodel(train_loader=train_loader_unbalanced, val_loader=val_loader_unbalanced, epochs_val=4,seed_val=2020,device=device,lr_value=5e-5)

===========Starting Epoch 1 / 4 ==============
Training starts

  Average training loss: 0.59

And now, validation STARTS
  Accuracy: 0.79
  Accuracy: 0.67
  Validation Loss: 0.61
===========Starting Epoch 2 / 4 ==============
Training starts

  Average training loss: 0.53

And now, validation STARTS
  Accuracy: 0.81
  Accuracy: 0.69
  Validation Loss: 0.64
===========Starting Epoch 3 / 4 ==============
Training starts

  Average training loss: 0.45

And now, validation STARTS
  Accuracy: 0.83
  Accuracy: 0.72
  Validation Loss: 0.62
===========Starting Epoch 4 / 4 ==============
Training starts

  Average training loss: 0.32

And now, validation STARTS
  Accuracy: 0.82
  Accuracy: 0.71
  Validation Loss: 0.77

Done !


In [0]:
import pandas as pd

In [28]:
df_stats = pd.DataFrame(data=results_unbalanced)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Valid F1
epoch,,,,
1,0.593989,0.608205,0.787755,0.670748
2,0.525662,0.642845,0.812245,0.691156
3,0.452452,0.616325,0.830612,0.717007
4,0.318853,0.771008,0.824490,0.710204


In [29]:
results_balanced = train_val_gendermodel(train_loader=train_loader_balanced, val_loader=val_loader_balanced, epochs_val=4,seed_val=2020,device=device,lr_value=5e-5)

===========Starting Epoch 1 / 4 ==============
Training starts

  Average training loss: 0.69

And now, validation STARTS
  Accuracy: 0.68
  Accuracy: 0.59
  Validation Loss: 0.89
===========Starting Epoch 2 / 4 ==============
Training starts

  Average training loss: 0.66

And now, validation STARTS
  Accuracy: 0.77
  Accuracy: 0.69
  Validation Loss: 0.52
===========Starting Epoch 3 / 4 ==============
Training starts

  Average training loss: 0.54

And now, validation STARTS
  Accuracy: 0.74
  Accuracy: 0.67
  Validation Loss: 0.74
===========Starting Epoch 4 / 4 ==============
Training starts

  Average training loss: 0.35

And now, validation STARTS
  Accuracy: 0.76
  Accuracy: 0.67
  Validation Loss: 1.10

Done !


In [30]:
df_stats = pd.DataFrame(data=results_balanced)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Valid F1
epoch,,,,
1,0.690740,0.886900,0.681633,0.585034
2,0.659529,0.520504,0.771429,0.688435
3,0.541589,0.739033,0.744898,0.672109
4,0.353204,1.095774,0.755102,0.670748


In [31]:
results_balanced_split = train_val_gendermodel(train_loader=train_loader_balanced_split, val_loader=val_loader_balanced_split, epochs_val=4,seed_val=2020,device=device,lr_value=5e-5)

===========Starting Epoch 1 / 4 ==============
Training starts

  Average training loss: 0.70

And now, validation STARTS
  Accuracy: 0.74
  Accuracy: 0.68
  Validation Loss: 0.69
===========Starting Epoch 2 / 4 ==============
Training starts

  Average training loss: 0.56

And now, validation STARTS
  Accuracy: 0.77
  Accuracy: 0.73
  Validation Loss: 0.85
===========Starting Epoch 3 / 4 ==============
Training starts

  Average training loss: 0.41

And now, validation STARTS
  Accuracy: 0.80
  Accuracy: 0.74
  Validation Loss: 0.85
===========Starting Epoch 4 / 4 ==============
Training starts

  Average training loss: 0.20

And now, validation STARTS
  Accuracy: 0.81
  Accuracy: 0.75
  Validation Loss: 1.00

Done !


In [32]:
df_stats = pd.DataFrame(data=results_balanced_split)
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Valid F1
epoch,,,,
1,0.695822,0.690138,0.737903,0.677419
2,0.556895,0.851728,0.768145,0.729839
3,0.412280,0.850640,0.796371,0.736559
4,0.201848,1.000318,0.808468,0.752688


Analyse

#### Saving the model ?

## Evaluation and qualitative analysis 

We train the model just as before with the attributes and samples we have chosen. In order to increase as much as possible the performance of the model we decide to train on both train and test data in this case. We will keep our development sample in order to make our qualitative analysis. 

In [0]:
# Let's take a balanced sample 
df_m = df.loc[df['sexe'] == 0]
df_f = df.loc[df['sexe'] == 1] 
df_m = df_m[0:len(df_f)]
df_eval = df_f.append(df_m)

In [0]:
#Shuffle the data
df_eval=df_eval.sample(frac=1).reset_index()

In [0]:
from itertools import repeat
n=2500
chunks, label_split,index_df=[],[],[]
j=0
for text in df_eval.Texte :
    txt=[text[i:i+n] for i in range(0, len(text), n)]
    chunks.append(txt)
    label_split.extend(repeat(df_eval.sexe[j], len(txt)))
    index_df.extend(repeat(df_eval.index[j], len(txt)))
    j+=1

chunks = [item for sublist in chunks for item in sublist]
df_eval=pd.DataFrame([chunks,label_split,index_df]).transpose()
df_eval.columns=['Texte','sexe','index']
df_eval['sexe'] = df_eval['sexe'].astype(int)


In [0]:
# Shuffle this again 
df_eval=df_eval.sample(frac=1).reset_index()

In [84]:
len_train = round(0.98*len(df_eval))
len_dev = len(df_eval) - len_train
len_train, len_dev

(12388, 253)

In [85]:
df_balanced_split= df_eval[0:len_train]
dev_balanced_split=df_eval[len_train:len(df_eval)]

# Report the number of speeches in the corpus.
print('Number of text in this balanced splitted corpus : {:,}\n'.format(df_balanced_split.shape[0]))
print('Number of text in the development sample : {:,}\n'.format(dev_balance_split.shape[0]))

Number of text in this balanced splitted corpus : 12,388

Number of text in the development sample : 253



In [86]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

texts = df_eval.Texte.values
labels = df_eval.sexe.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
num_truncated_tokens =[]
# Apply function to our corpus
for text in texts:
    encoded_dict = tokenizer.encode_plus(
                        text,                      # text
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 500,           # We choose for now a max length of 500.
                        pad_to_max_length = True,    # Pad text to max (marche pas en pad left ?)
                        return_attention_mask = True,   # Construct attention masks
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens =True, # return overflowing token information
                  )
    
    # Map tokens to their id in the dictionnary 
    # We add this to our list    
    input_ids.append(encoded_dict['input_ids'])

    #num_truncated_tokens.append(encoded_dict['num_truncated_tokens'])
    
    # 3. Attention masks
    attention_masks.append(encoded_dict['attention_mask'])

# We convert all this into tensors in order to be able to make it work on GPU 
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Original text and transformed tensor print 
print("Let's check for the first text indexes, attention masks and labels")
print(" ")
print('Original: ', texts[0][0:100])
print('IDs:', input_ids[0][0:100])
print('Attention masks:', attention_masks[0][0:100])
print('labels',labels[0])


# Combine all above
dataset = TensorDataset(input_ids, attention_masks, labels)

# We set the size of the batch ( usually set around 16 or 32), we will take the lower bound because of the large text length
batch_size = 10

# We create data loaders for the train and validation dataset. 
train_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = SequentialSampler(dataset), 
            batch_size = 10 # Trains with this batch size.
        )

print('Data loaders created for train [0]')

Let's check for the first text indexes, attention masks and labels
 
Original:  membres dans ce domaine et que ce critère soit mieux pondéré dans la répartition proposée.Le disposi
IDs: tensor([    5,   580,    29,    44,   813,    14,    27,    44, 11094,   191,
          334, 31308,    29,    13,  5443,  6012,     9,   990,  2507,   279,
           98, 11439,    37,    19,   214,  9099,     8,    17,    11,   974,
         1467,     7,    22,   770,    13,  1671,   192,    16,  3893,     8,
         7809,     7,    42,    17,    11,  3275,   360,  4749,  7633,   449,
         3701, 10669,     7,    24,   745,    63, 15022,    23,  3429,    25,
          960,     8,   548,    20,  5542, 14784,     7,   289,     8,   351,
         6442,    22,  1118,   806,    14,     7,    36,   394,     7,    29,
           19,  2775,  9965, 23459,  1727,     9,  1855,  1817,     7,    14,
           50,    17,    11,    73,  2410,     7,    13,  3599,   279,    98])
Attention masks: tensor([1, 1, 1,

In [0]:
def prepare_to_feed_train(df,length,batch_size_value):
  from torch.utils.data import TensorDataset, random_split
  from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

  texts = df_eval.Texte.values
  labels = df_eval.sexe.values

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []
  num_truncated_tokens =[]
  # Apply function to our corpus
  for text in texts:
      encoded_dict = tokenizer.encode_plus(
                          text,                      # text
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = length,           # We choose for now a max length of 500.
                          pad_to_max_length = True,    # Pad text to max (marche pas en pad left ?)
                          return_attention_mask = True,   # Construct attention masks
                          return_tensors = 'pt',     # Return pytorch tensors.
                          return_overflowing_tokens =True, # return overflowing token information
                    )
      
      # Map tokens to their id in the dictionnary 
      # We add this to our list    
      input_ids.append(encoded_dict['input_ids'])
  
      #num_truncated_tokens.append(encoded_dict['num_truncated_tokens'])
      
      # 3. Attention masks
      attention_masks.append(encoded_dict['attention_mask'])

  # We convert all this into tensors in order to be able to make it work on GPU 
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)

  # Original text and transformed tensor print 
  print("Let's check for the first text indexes, attention masks and labels")
  print(" ")
  print('Original: ', texts[0][0:100])
  print('IDs:', input_ids[0][0:100])
  print('Attention masks:', attention_masks[0][0:100])
  print('labels',labels[0])


  # Combine all above
  dataset = TensorDataset(input_ids, attention_masks, labels)

  # We create data loaders for the train and validation dataset. 
  train_dataloader = DataLoader(
              dataset,  # The training samples.
              sampler = SequentialSampler(dataset), 
              batch_size = batch_size_value # Trains with this batch size.
          )
  
  print('Data loaders created for train [0]')

  return train_dataloader

In [79]:
train_loader_balanced_split= prepare_to_feed_train(df_balanced_split,length=500,batch_size_value=1)

Let's check for the first text indexes, attention masks and labels
 
Original:  mptueux, mais l'enjeu qui sous-tend notre action est de ce niveau et nous devons en être conscients.
IDs: tensor([    5, 13393,   744,   914,     7,    65,    17,    11, 12854,    31,
          161,    26, 13721,   127,  1703,    30,     8,    44,   359,    14,
           63,  4092,    22,    98, 19531,     9,  1855,  3606,    63,   296,
         9009,    28,  2131,    29,    13,  1671,   192,    13,  1596,    15,
         5360,   360,  9249,    55,     7,   182,    40,     8,   301,   250,
            8,  1671, 17175,    35,     9,   232,  1662,   711, 10013,   194,
           13, 11908,     8,   127, 28651,    24,    17,    11,  4378,     8,
           13,  2695,  6901,  7305,    29,    17,    11, 15184,    25,   499,
         2028,     9, 28148,     7,    15,   300,     7,   298,     8,    13,
        20253,  2139,     7,    63,   296,   523,    23, 11263,    29,    13])
Attention masks: tensor([1, 1, 1,

In [88]:
############################  IMPORT MODEL ################################################
gender_model = CamembertForSequenceClassification.from_pretrained(
"camembert-base", 
num_labels = 2, # We have two different labels Women = 1 and Men =0   
output_attentions = False, 
output_hidden_states = False, )

gender_model.cuda()

############################## RANDOM SEED ##################################################

import random
# Let's put a seed to make this result reproducible 
seed=2020
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

############################### LEARNING RATE SCHEDULER #######################################

# https://huggingface.co/transformers/migration.html 
# https://pytorch.org/docs/stable/optim.html (default values)

import torch.nn as nn
import torch.optim as optim
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

epochs = 1  # In order to fine tune our model we will first set the number of epochs to 4.

# We choose Binary cross enthropy with logits loss for the loss computation. It seems to be the most adapted loss to our problem. 
criterion = nn.BCEWithLogitsLoss()
#Implements Adam algorithm with weight decay fix.
opti = AdamW(gender_model.parameters(),
                lr =5e-5, # learning rate (default = 1e-3)
                eps = 1e-8 # prevents division by 0 (default = 1e-8)
              )

num_training_steps = len(train_loader_balanced_split) * epochs
# Create a schedule with a learning rate that decreases linearly after linearly increasing during a warmup period (0 here)
scheduler = get_linear_schedule_with_warmup(opti, 
                                          num_warmup_steps = 0,
                                          num_training_steps = num_training_steps)


# We want to evaluate the training phase 
training_stats = []

for ep in range(0, epochs):
  print('===========Starting Epoch {} / {} =============='.format(ep+1,epochs))
  print('Training starts')

  ################################### TRAINING ################################

  #Put the model in training mode
  gender_model.train()

  # Set the train loss for the epoch to 0 
  total_train_loss = 0

  for step, batch in enumerate(train_loader_balanced_split):
    # Clear gradients 
    gender_model.zero_grad() # (opti.zerograd ? )

    # Cpy the 3 batch to GPU 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    #return loss and logits
    loss, logits = gender_model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels) 
    
    # Accumulate training loss for all batches 
    total_train_loss += loss.item()

    #Backpropagating the gradients 
    loss.backward()

    # Prevent exploding gradients problem  (forcing the gradients to be small, the parameter updates will not push the parameters too far from their previous values)
    torch.nn.utils.clip_grad_norm_(gender_model.parameters(), 1.0)

    # Update parameters 
    opti.step()

    # Update learning rate schedule
    scheduler.step()

#Calculate the average training loss over all batches  
avg_train_loss = total_train_loss / len(train_loader_balanced_split)
print("")
print("  Average training loss: {0:.2f}".format(avg_train_loss))
print('')  

===========Starting Epoch 1 / 1 ==============
Training starts

  Average training loss: 0.70



Preparing development sequence 

In [0]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


texts = dev_balanced_split.Texte.values
labels = dev_balanced_split.sexe.values

In [0]:

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
num_truncated_tokens =[]
original_text =[]
tokenized_text=[]

for text in texts:
  original_text.append(text)
  tokenized_text.append(tokenizer.tokenize(text))
  encoded_dict = tokenizer.encode_plus(
                        text,                      # text
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 500,           # We choose for now a max length of 500.
                        pad_to_max_length = True,    # Pad text to max (marche pas en pad left ?)
                        return_attention_mask = True,   # Construct attention masks
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens =True, # return overflowing token information
                  )
    
    # Map tokens to their id in the dictionnary 
    # We add this to our list    
  input_ids.append(encoded_dict['input_ids'])

    #num_truncated_tokens.append(encoded_dict['num_truncated_tokens'])
    
    # 3. Attention masks
  attention_masks.append(encoded_dict['attention_mask'])

In [0]:

# We convert all this into tensors in order to be able to make it work on GPU 
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [95]:
# Original text and transformed tensor print 
print("Let's check for the first text indexes, attention masks and labels")
print(" ")
print('Original: ', texts[0][0:100])
print('Tokenized: ', tokenized_text[0])
print('IDs:', input_ids[0][0:100])
print('Attention masks:', attention_masks[0][0:100])
print('labels',labels[0])

Let's check for the first text indexes, attention masks and labels
 
Original:  amitié et un profond respect celles et ceux qui sont présents dans cette salle. Mes chers amis, vous
Tokenized:  ['▁amitié', '▁et', '▁un', '▁profond', '▁respect', '▁celles', '▁et', '▁ceux', '▁qui', '▁sont', '▁présents', '▁dans', '▁cette', '▁salle', '.', '▁Mes', '▁chers', '▁amis', ',', '▁vous', '▁le', '▁verrez', ',', '▁ce', '▁film', '▁vous', '▁donne', '▁la', '▁parole', '▁pour', '▁aborder', '▁la', '▁fin', '▁des', '▁camps', ',', '▁la', '▁libération', '▁et', '▁les', '▁conditions', '▁peu', '▁connues', ',', '▁souvent', '▁tu', 'es', ',', '▁de', '▁votre', '▁ré', 'adaptation', '▁à', '▁la', '▁vie', '▁hors', '▁des', '▁camps', ',', '▁vous', '▁que', '▁la', '▁barbarie', '▁na', 'zie', '▁avait', '▁condamné', '▁à', '▁la', '▁disparition', ',', '▁dans', '▁la', '▁nuit', '▁et', '▁le', '▁brouillard', ',', '▁vous', '▁dont', '▁l', "'", 'extermination', '▁avait', '▁été', '▁méthodique', 'ment', '▁planifié', 'e', ',', '▁vous', '▁qui'

In [0]:
# Combine all above
dataset = TensorDataset(input_ids, attention_masks, labels)
batch_size = 1
# We create data loaders for the train and validation dataset. 
dev_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = SequentialSampler(dataset), # We set to sequential and we keep track 
            batch_size = batch_size # Trains with this batch size.
        )

In [0]:
# Put model in evaluation mode 
gender_model.eval()

total_pred,total_labels, total_eval_loss,scores_max,scores_min=[],[], [],[],[]
for batch in dev_dataloader:
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  
  # We don't care about gradients for eval

  with torch.no_grad(): 
    (loss, logits) = gender_model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
  total_eval_loss.append(loss.item())

    # Move logits and labels to CPU 
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  score_max = np.amax(logits, axis=1).flatten().item()
  score_min = np.amin(logits, axis=1).flatten().item()
  pred_flat = np.argmax(logits, axis=1).flatten().item()
  labels_flat = label_ids.flatten().item()

  total_pred.append(pred_flat)
  total_labels.append(labels_flat)
  scores_max.append(score_max)
  scores_min.append(score_min)


In [0]:
results_dev=pd.DataFrame([total_labels,total_pred,scores_max,scores_min]).transpose()
results_dev.columns=['returned_labels','model_pred','scores_max','score_min']

,level_0,Texte,sexe,index
12388,2457,amitié et un profond respect celles et ceux qu...,0,499
12389,11792,"américains prônent la liberté, mais cela ne l...",0,2323
12390,4140,doit donc être accepté socialement. Nous voyo...,1,838
12391,3018,"Mesdames, Messieurs,Le sujet que vous avez abo...",1,612
12392,252,Ce n'est pas sur des déclarations d'intention ...,0,54
...,...,...,...,...
12636,1152,"nt à travers moi vous le confirme, l'Accord de...",1,242
12637,10108,"Monsieur le Préfet, Mesdames et messieurs les...",1,1993
12638,1363,ance contribue à la construction des deux cent...,0,275
12639,5755,"gagement financier de l'Etat, un engagement de...",0,1159


In [0]:
frames = [dev_balanced_split[['Texte','sexe','index']].reset_index(), results_dev]
result = pd.concat(frames,axis=1)

In [111]:
result.head(2)

,level_0,Texte,sexe,index,returned_labels,model_pred,scores_max,score_min
0,12388,amitié et un profond respect celles et ceux qu...,0,499,0.0,0.0,1.863802,-1.760256
1,12389,"américains prônent la liberté, mais cela ne l...",0,2323,0.0,0.0,2.099839,-2.016395


In [112]:
result[result.model_pred!=result.sexe].head(2)

,level_0,Texte,sexe,index,returned_labels,model_pred,scores_max,score_min
2,12390,doit donc être accepté socialement. Nous voyo...,1,838,1.0,0.0,1.387547,-1.239461
10,12398,êtres humains que comme des variables d'ajuste...,1,1749,1.0,0.0,1.835730,-1.751564


We want to dive a bit into the model and see how it makes a choice and why it fails on thos 38 sentences. Let's take one of them. We will redo point 4 of TD4 to see the score reached by each word. 

In [0]:
max_score_fail = result[result.model_pred!=result.sexe].scores_max.max()
sentence_to_analyse = result[result.scores_max==max_score_fail]
sentence_to_analyse = sentence_to_analyse[['Texte','sexe']]

In [114]:
sentence_to_analyse['Texte']

167    coup plus grave.Je ne parlerai pas du bilan de...
Name: Texte, dtype: object

In [0]:
for text in sentence_to_analyse['Texte']:
  tokens = text.split('.')

In [0]:
sentences_to_analyse =pd.DataFrame(tokens)
sentences_to_analyse=sentences_to_analyse.assign(sexe=1)
sentences_to_analyse.columns=['Texte','sexe']


In [117]:
texts

array(['amitié et un profond respect celles et ceux qui sont présents dans cette salle. Mes chers amis, vous le verrez, ce film vous donne la parole pour aborder la fin des camps, la libération et les conditions peu connues, souvent tues, de votre réadaptation à la vie hors des camps, vous que la barbarie nazie avait condamné à la disparition, dans la nuit et le brouillard, vous dont l\'extermination avait été méthodiquement planifiée, vous qui étiez presque, déjà, hors du monde des vivants." Personne n\'avait envie de nous entendre "." Durant les premiers jours qui ont suivi notre retour, nous voulions parler, être entendus []. Nous ramenions avec nous notre mémoire, notre expérience toute vivante. Nous éprouvions un désir frénétique de la dire telle quelle []. Et cependant c\'était impossible [] "" A nous-mêmes, ce que nous avions à dire, commençait à nous paraître inimaginable "Mesdames et Messieurs, ces témoignages, cette incompréhension bouleversante, cette " mémoire blessée ", vo

In [0]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


texts = sentences_to_analyse.Texte.values
labels = sentences_to_analyse.sexe.values

In [0]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
num_truncated_tokens =[]
original_text =[]
tokenized_text=[]

for text in texts:
  original_text.append(text)
  tokenized_text.append(tokenizer.tokenize(text))
  encoded_dict = tokenizer.encode_plus(
                        text,                      # text
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 50,           # We choose for now a max length of 500.
                        pad_to_max_length = True,    # Pad text to max (marche pas en pad left ?)
                        return_attention_mask = True,   # Construct attention masks
                        return_tensors = 'pt',     # Return pytorch tensors.
                        return_overflowing_tokens =True, # return overflowing token information
                  )
    
    # Map tokens to their id in the dictionnary 
    # We add this to our list    
  input_ids.append(encoded_dict['input_ids'])

    #num_truncated_tokens.append(encoded_dict['num_truncated_tokens'])
    
    # 3. Attention masks
  attention_masks.append(encoded_dict['attention_mask'])

In [0]:
# We convert all this into tensors in order to be able to make it work on GPU 
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [0]:
# Combine all above
batch_size = 1
dataset = TensorDataset(input_ids, attention_masks, labels)
# We create data loaders for the train and validation dataset. 
dev_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = SequentialSampler(dataset), # We set to sequential and we keep track 
            batch_size = batch_size # Trains with this batch size.
        )

In [0]:
# Put model in evaluation mode 
gender_model.eval()

total_pred,total_labels, total_eval_loss,scores_max,scores_min=[],[], [],[],[]
for batch in dev_dataloader:
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  
  # We don't care about gradients for eval

  with torch.no_grad(): 
    (loss, logits) = gender_model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask,
                                labels=b_labels)
  total_eval_loss.append(loss.item())

    # Move logits and labels to CPU 
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  score_max = np.amax(logits, axis=1).flatten().item()
  score_min = np.amin(logits, axis=1).flatten().item()
  pred_flat = np.argmax(logits, axis=1).flatten().item()
  labels_flat = label_ids.flatten().item()

  total_pred.append(pred_flat)
  total_labels.append(labels_flat)
  scores_max.append(score_max)
  scores_min.append(score_min)


In [0]:
results_dev=pd.DataFrame([total_labels,total_pred,scores_max,scores_min]).transpose()
results_dev.columns=['returned_labels','model_pred','scores_max','score_min']

In [0]:
frames = [sentences_to_analyse.reset_index(), results_dev]
result = pd.concat(frames,axis=1)

In [126]:
result

,index,Texte,sexe,returned_labels,model_pred,scores_max,score_min
0,0,coup plus grave,1,1.0,0.0,1.009956,-0.874495
1,1,Je ne parlerai pas du bilan de Nicolas Sarkozy,1,1.0,0.0,1.301345,-1.258373
2,2,Juste un mot : il a dit quand on veut on peut...,1,1.0,0.0,1.463775,-1.332182
3,3,Il n'y a rien eu de tout cela pendant la prési...,1,1.0,0.0,1.462664,-1.375385
4,4,"Et en tout cas, les autres pays ont pu lui di...",1,1.0,0.0,1.705040,-1.593131
5,5,"Puis il faut le dire, en rentrant dans l'OTAN ...",1,1.0,0.0,1.778462,-1.668161
6,6,Voilà ce bilan,1,1.0,1.0,0.225109,-0.100552
7,7,Il a dit : je veux être jugé sur mes résultats,1,1.0,0.0,1.946002,-1.854517
8,8,"Et bien le 7 juin, on va le juger sur ses rés...",1,1.0,0.0,1.300824,-1.161809
9,9,"Alors oui, l'Europe mérite ce grand débat",1,1.0,0.0,0.117594,-0.090714


In [125]:
result[result.scores_max>=1].Texte

0                                       coup plus grave
1        Je ne parlerai pas du bilan de Nicolas Sarkozy
2      Juste un mot : il a dit quand on veut on peut...
3     Il n'y a rien eu de tout cela pendant la prési...
4      Et en tout cas, les autres pays ont pu lui di...
5     Puis il faut le dire, en rentrant dans l'OTAN ...
7        Il a dit : je veux être jugé sur mes résultats
8      Et bien le 7 juin, on va le juger sur ses rés...
10     Et aujourd'hui, comme d'habitude, quand on ne...
11    J'aimerais rappeler que l'Europe est née des p...
12     Or aujourd'hui, on s'adresse à leur moelle ép...
13     Mais aujourd'hui, ça ne marchera pas, car nou...
15    Et voyez, il y a quelques jours, des bandes qu...
16     Un enfant qui donne un coup de poignard à un ...
17     Des prisonniers qui sont dans la nature : que...
18     Elle a cassé la police de proximité, et voudr...
19    On a supprimé les éducateurs, les accompagnate...
20     Qu'on ne nous donne pas de leçons, qu'on 